<a href="https://colab.research.google.com/github/peppopi/Piratas/blob/main/Recorda%20Poner%20tus%20imagenes%20en%20la%20carpeta%20creada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
 #@title ## 🏴‍☠️ ¡Bienvenido a bordo, marinero!
#@markdown Ahoy, marinero intrépido, ¡bienvenido al inicio de una gran aventura! Prepárate para surcar los mares virtuales y descubrir los tesoros escondidos en tus imágenes. Con el poder de la inteligencia artificial y nuestra astucia pirata, desentrañaremos los secretos que se ocultan en cada pixel. ¡Afilen sus espadas y preparen los cañones, porque una experiencia inolvidable nos espera en este viaje hacia lo desconocido! 🏴‍☠️🌊

#@markdown En esta celda, prepararemos nuestro barco y crearemos las carpetas necesarias en tu Google Drive para comenzar nuestra emocionante travesía. ¡Prepárate para zarpar! ⚓️
COLAB = True

if COLAB:
    from google.colab.output import clear as clear_output
else:
    from IPython.display import clear_output

nombre_proyecto = "EeveelutionsCollection" #@param {type:"string"}
project_name = nombre_proyecto.strip()

def create_folders():
    if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
        print("¡Ahoy, capitán! Por favor, elige un nombre de proyecto válido. Evitemos los espacios y los caracteres especiales en este viaje.")
    else:
        if COLAB and not os.path.exists('/content/drive'):
            from google.colab import drive
            print("⚓️ Conectando con Google Drive... Espérame, ¡no puedo resistir la emoción!")
            drive.mount('/content/drive')

        main_dir = "/content/Dataset" #@markdown Avast, marinero valiente! Antes de que nos adentremos en esta emocionante travesía, necesitamos establecer una ruta principal donde guardaremos todos los tesoros que encontraremos en el camino. Elige sabiamente la ubicación en la que deseas almacenar tus descubrimientos. Recuerda, una buena ruta es la clave para mantener tus tesoros seguros y organizados. ¡Adelante, capitán intrépido, dinos dónde quieres que depositemos tus preciadas recompensas! ⚓️💎
        images_folder = os.path.join(main_dir, project_name)

        if os.path.exists(images_folder):
            print(f"¡Ahoy, marinero! Ya tienes la carpeta {images_folder} preparada para guardar tus imágenes. ¡Excelente trabajo!")
        else:
            os.makedirs(images_folder)
            print(f"⚓️ ¡Carpetas creadas! La carpeta para las imágenes está en {images_folder}. ¡Preparados para llenarla de tesoros!")

create_folders()


In [ ]:
import os


#@markdown ### ⚡️ Descubre los secretos de las imágenes
#@markdown ¡Preparaos, marineros, para desvelar los tesoros ocultos en vuestras imágenes! Usaremos la magia de la inteligencia artificial para explorar el vasto océano del anime y los misterios del fotorealismo. Con el poderoso Waifu Diffusion, desentrañaremos las etiquetas y los tesoros escondidos en cada imagen animada. Y para las maravillas del fotorealismo, contaremos con BLIP, quien nos revelará los subtítulos y las leyendas que aguardan en las profundidades visuales. ¡Estos tesoros visuales potenciarán la grandeza de tu Lora de una manera espectacular! Prepárate para zarpar y descubrir los secretos que aguardan en cada imagen.

metodo = "\uD83D\uDCF7 Fotorealismo"  #@param ["⚔️ Anime", "📷 Fotorealismo"]
method = metodo
#@markdown **⚔️ Anime:** El umbral marca el nivel de certeza que debe alcanzar la IA para asignar cada tag. ¡A menor umbral, más tags conseguiremos! Recomiendo mayor umbral para navegar en aguas turbulentas , ¡capitán!
umbral = 0.43  #@param {type:"slider", min:0.0, max:1.0, step:0.01}
tag_threshold = umbral
#@markdown **🏴‍☠️ Establece los límites de cantidad de tesoros encontrados antes de regresar a puerto, arrr!** Modifica estos valores a tu gusto y prepárate para desenterrar una gran cantidad de tesoros ocultos. ¡Recuerda, cuanto más amplios sean los límites, más riquezas descubriremos en nuestro viaje! Ajusta los números a tu antojo y prepárate para izar las velas hacia lo desconocido.
largo_minimo = 10  #@param {type:"number"}
caption_min = largo_minimo
largo_maximo = 100  #@param {type:"number"}
caption_max = largo_maximo

kohya = "/content/kohya-trainer"

if not os.path.exists(kohya):
    !git clone https://github.com/kohya-ss/sd-scripts {kohya}

if "⚔️ Anime" in method:
    if "step4a_installed_flag" not in globals():
        print("🏴‍☠️ Izando las velas y afilando los ganchos...\n")
        !pip -q install tensorflow==2.12.0 huggingface-hub==0.12.0 accelerate==0.15.0 transformers==4.26.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6 torchvision albumentations
        if not get_ipython().__dict__['user_ns']['_exit_code']:
            clear_output()
            step4a_installed_flag = True
        else:
            print("❌ ¡A la bodega! ¡Error en la instalación! Pero seguiremos adelante, ¡arrr!")

    print("🚶‍♂️ ¡Sube a bordo, marinero! ¡Izamos anclas y comenzamos la travesía!\n")

    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
    %env PYTHONPATH={kohya}
    !python {kohya}/finetune/tag_images_by_wd14_tagger.py \
        {images_folder} \
        --repo_id=SmilingWolf/wd-v1-4-swinv2-tagger-v2 \
        --model_dir={root_dir} \
        --thresh={tag_threshold} \
        --batch_size=8 \
        --caption_extension=.txt \
        --force_download

    if not get_ipython().__dict__['user_ns']['_exit_code']:
        print("🦜 ¡Excelente! Hemos encontrado verdaderos tesoros en tus imágenes. Pero antes de partir, ajustemos los detalles finales. Quitaré los guiones bajos de los tags y los prepararé para su uso.")

        from collections import Counter
        top_tags = Counter()
        for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
            with open(os.path.join(images_folder, txt), 'r') as f:
                tags = [t.strip() for t in f.read().split(",")]
                tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
            top_tags.update(tags)
            with open(os.path.join(images_folder, txt), 'w') as f:
                f.write(", ".join(tags))

        clear_output()
        print("📊 ¡Tesoro encontrado! Aquí están las 50 etiquetas más comunes en tus imágenes:")
        print("\n".join(f"🏴‍☠️ {k} ({v})" for k, v in top_tags.most_common(50)))

else:  # 📷 Fotorealismo
    if "step4b_installed_flag" not in globals():
        print("🏴‍☠️ Izando las velas y afilando los ganchos...\n")
        !pip -q install timm==0.6.12 fairscale==0.4.13 transformers==4.26.0 requests==2.28.2 accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6
        if not get_ipython().__dict__['user_ns']['_exit_code']:
            clear_output()
            step4b_installed_flag = True
        else:
            print("❌ ¡A la bodega! ¡Error en la instalación! Pero seguiremos adelante, ¡arrr!")

    print("🚶‍♂️ ¡Sube a bordo, marinero! ¡Izamos anclas y comenzamos la travesía!\n")

    os.chdir(kohya)
    %env PYTHONPATH={kohya}
    !python {kohya}/finetune/make_captions.py \
        {images_folder} \
        --beam_search \
        --max_data_loader_n_workers=2 \
        --batch_size=8 \
        --min_length={caption_min} \
        --max_length={caption_max} \
        --caption_extension=.txt

    if not get_ipython().__dict__['user_ns']['_exit_code']:
        import random
        captions = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
        sample = []
        for txt in random.sample(captions, min(10, len(captions))):
            with open(os.path.join(images_folder, txt), 'r') as f:
                sample.append(f.read())

        os.chdir(root_dir)
        %env PYTHONPATH=/env/python
        clear_output()
        print("📊 ¡Tesoro encontrado! Aquí tienes algunos ejemplos de los subtítulos en tus imágenes:")
        print("\n".join([f"🏴‍☠️ {s}\n" for s in sample]))

print("\n🏴‍☠️ ¡La búsqueda ha concluido, marinero! Hemos terminado de describir tus imágenes, y ahora está en tus manos usar este valioso conocimiento para el bien de tu Lora. ¡A seguir navegando y descubriendo nuevos tesoros visuales!")


In [ ]:
import os
#@title ## 🏴‍☠️ ¡Se fue todo a la mierda, caiptan!
print("🏴‍☠️ Preparándonos para zarpar en busca de las últimas versiones...")
print("🌊 Navegando por los mares de la actualización de bibliotecas...")

# Aquí viene la parte divertida del "pirata" que anima a los alumnos a actualizar las bibliotecas
print("⚓️ ¡Argh, marineros! Si queréis las últimas versiones, necesitáis actualizar las bibliotecas. A continuación, os indicaré qué partes debéis modificar:")
print("O si claro, es la que esta dando Error") 

print("⚙️ ¡Primero, aseguraos de que estáis en el lugar correcto!")
print("   ⚠️ Antes de continuar, por favor, verificad que estáis en el entorno adecuado. ¡No queremos navegar por aguas equivocadas!")
print("   👉 Revisad y actualizad la variable `nombre_proyecto` con la ruta correcta de vuestro proyecto.")

print("🔧 ¡A continuación, mostraremos cómo actualizar las bibliotecas!")
print("   📦 Buscad las líneas donde se instalan las bibliotecas y cambiad las versiones por las más recientes.")
print("   👉 Por ejemplo, si veis `timm==0.6.12`, cambiadlo a `timm==0.7.0`.")
print("   👉 Haced esto con todas las bibliotecas que deseéis actualizar.")
print("   📝 Nota: ¡Aseguraos de usar las versiones compatibles con vuestro proyecto!")
print()

print("🏴‍☠️ ¡Yarr, habéis terminado! Ahora podéis disfrutar de las últimas mejoras y funciones en vuestras aventuras de etiquetado de imágenes.")